# TO BE FIXED / COMPLETED!!!!

In [51]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

### 0. Cleanup Mesh

In [33]:
def remove_redundant_vertices(mesh):
    # Get the vertices and triangles
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    
    # Identify unique vertex indices used in the triangles
    unique_vertex_indices = np.unique(triangles)
    
    # Create a mapping from old vertex indices to new ones
    old_to_new_indices = {old_idx: new_idx for new_idx, old_idx in enumerate(unique_vertex_indices)}
    
    # Create a new list of vertices that are only the used ones
    new_vertices = vertices[unique_vertex_indices]
    
    # Update triangle indices to the new vertex indices
    new_triangles = np.array([[old_to_new_indices[old_idx] for old_idx in triangle] for triangle in triangles])
    
    # Create a new mesh with the cleaned vertices and updated triangles
    cleaned_mesh = o3d.geometry.TriangleMesh()
    cleaned_mesh.vertices = o3d.utility.Vector3dVector(new_vertices)
    cleaned_mesh.triangles = o3d.utility.Vector3iVector(new_triangles)
    
    # Copy colors, normals, etc. if they exist
    if mesh.has_vertex_colors():
        cleaned_mesh.vertex_colors = o3d.utility.Vector3dVector(np.asarray(mesh.vertex_colors)[unique_vertex_indices])
    if mesh.has_vertex_normals():
        cleaned_mesh.vertex_normals = o3d.utility.Vector3dVector(np.asarray(mesh.vertex_normals)[unique_vertex_indices])
    
    return cleaned_mesh

In [34]:
# Load the mesh
intraoral_mesh = o3d.io.read_triangle_mesh(r'D:\sunny\Codes\DPS\data_teethseg\origin\000101_origin.ply')
# print(np.array(mesh.vertices).shape)
# print(np.array(mesh.triangles).shape)

# Remove redundant vertices
intraoral_mesh = remove_redundant_vertices(intraoral_mesh)
# print(np.array(mesh.vertices).shape)
# print(np.array(mesh.triangles).shape)

# Visualize the cleaned mesh
o3d.visualization.draw_geometries([intraoral_mesh])

### 1. Create Splines and Extrude to Projection Planes

Utility functions

In [45]:
def calculate_spline_points(mesh, n_points=8, n_spline_points=100):
    vertices = np.asarray(mesh.vertices)
    angles = np.linspace(0, 2 * np.pi, n_points, endpoint=False)
    
    min_points = []
    max_points = []
    
    for theta in angles:
        direction = np.array([np.cos(theta), 0, np.sin(theta)])
        
        projections = vertices @ direction
        min_index = np.argmin(projections)
        max_index = np.argmax(projections)
        
        min_points.append(vertices[min_index, [0, 2]])
        max_points.append(vertices[max_index, [0, 2]])
    
    min_points = np.array(min_points)
    max_points = np.array(max_points)
    
    # Interpolate using cubic splines
    t = np.linspace(0, 2 * np.pi, n_points, endpoint=False)
    t_spline = np.linspace(0, 2 * np.pi, n_spline_points)
    
    spline_min_x = CubicSpline(t, min_points[:, 0])(t_spline)
    spline_min_z = CubicSpline(t, min_points[:, 1])(t_spline)
    spline_max_x = CubicSpline(t, max_points[:, 0])(t_spline)
    spline_max_z = CubicSpline(t, max_points[:, 1])(t_spline)
    
    spline_min_points = np.vstack((spline_min_x, np.zeros(n_spline_points), spline_min_z)).T
    spline_max_points = np.vstack((spline_max_x, np.zeros(n_spline_points), spline_max_z)).T

    return spline_min_points, spline_max_points


def create_spline_curve(points):
    return o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector([[i, (i + 1) % len(points)] for i in range(len(points))])
    )


In [46]:
def extrude_spline(points, extrusion_length_pos, extrusion_length_neg):
    extruded_points = []
    for y in [extrusion_length_neg, extrusion_length_pos]:
        extruded_points.append(np.hstack((points, np.full((points.shape[0], 1), y))))
    extruded_points = np.vstack(extruded_points)
    extruded_points = extruded_points[:, [0, 2, 1]] # swap back the y and z axis
    
    # Create triangular faces for the extrusion
    n_points = len(points)
    faces = []
    for i in range(n_points):
        next_i = (i + 1) % n_points # Ensure circular connection
        # Create two triangles for each face of the cylinder
        faces.append([i, next_i, next_i + n_points])
        faces.append([i, next_i + n_points, i + n_points])
    
    return extruded_points, faces

Create splines

In [47]:
# Calculate spline points
min_spline_points, max_spline_points = calculate_spline_points(intraoral_mesh)

# Create spline curves
min_spline_curve = create_spline_curve(min_spline_points)
max_spline_curve = create_spline_curve(max_spline_points)
print(min_spline_points)

[[-36.9082718   -9.35032041]
 [-31.47106017 -20.41068023]
 [ 27.53755381 -20.64838084]
 [ 32.00259003 -18.50693527]
 [ 36.98967021  -7.78544466]
 [ 15.42202642  23.19911091]
 [  4.37705733  28.24925648]
 [-14.71832703  23.55972164]]


Extrude to curve planes

In [48]:
# Calculate the extrusion length
y_min = np.min(np.asarray(intraoral_mesh.vertices)[:, 1])
y_max = np.max(np.asarray(intraoral_mesh.vertices)[:, 1])
extrusion_length_pos = y_max
extrusion_length_neg = y_min

# Extrude the spline curves
min_spline_extruded_points, min_spline_faces = extrude_spline(min_spline_points, extrusion_length_pos, extrusion_length_neg)
max_spline_extruded_points, max_spline_faces = extrude_spline(max_spline_points, extrusion_length_pos, extrusion_length_neg)

# Create meshes for the extruded splines
min_spline_extruded = o3d.geometry.TriangleMesh()
min_spline_extruded.vertices = o3d.utility.Vector3dVector(min_spline_extruded_points)
min_spline_extruded.triangles = o3d.utility.Vector3iVector(min_spline_faces)
min_spline_extruded.paint_uniform_color([0.5, 1.0, 0.5])

max_spline_extruded = o3d.geometry.TriangleMesh()
max_spline_extruded.vertices = o3d.utility.Vector3dVector(max_spline_extruded_points)
max_spline_extruded.triangles = o3d.utility.Vector3iVector(max_spline_faces)
max_spline_extruded.paint_uniform_color([0.5, 0.5, 1.0])


TriangleMesh with 16 points and 16 triangles.

Visualization

In [50]:
# Create coordinate axes
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])

o3d.visualization.draw_geometries([intraoral_mesh, min_spline_extruded, max_spline_extruded, coordinate_frame])